In [1]:
import torch
from modules.prenet import PreNet
from modules.encdec import get_encoder, get_gpt2_decoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/anaconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prenet = PreNet(
    input_dim=384,
    output_dim=768,
    bottleneck_dim=128,
    prefix_len=20
)
prenet.load_state_dict(torch.load('saved_models/prenet_prefix_tuning.pth', map_location=device))
prenet.to(device)

encoder = get_encoder("all-MiniLM-L6-v2")
decoder, tokenizer = get_gpt2_decoder()

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [3]:
def vec_to_text(embedding, decoder, tokenizer, prenet, gen_len=50):
    """
    Given input text, encode it, generate prefix via PreNet, and autoregressively decode output text.
    """
    decoder.eval()
    prenet.eval()
    with torch.no_grad():
        prefix = prenet(embedding.unsqueeze(0))  # (1, prefix_len, model_dim)

        generated = prefix  # initial embeddings
        generated_ids = []
        for _ in range(gen_len):
            outputs = decoder(inputs_embeds=generated)
            next_logits = outputs.logits[:, -1, :]
            next_id = torch.argmax(next_logits, dim=-1).unsqueeze(-1)  # greedy
            generated_ids.append(next_id)
            next_embed = decoder.transformer.wte(next_id)
            generated = torch.cat([generated, next_embed], dim=1)

    gen_ids = torch.cat(generated_ids, dim=1)
    return tokenizer.decode(gen_ids[0].cpu().numpy(), skip_special_tokens=True)

In [20]:
text = "Today I'll do some groceries."
embedding = encoder.encode(text, convert_to_tensor=True).to(device)

generated_text = vec_to_text(
    embedding, decoder, tokenizer, prenet, 50
)

print(generated_text)

 Go grab groceries today .                                             


In [10]:
text = "Hello, what's your name?"
embedding = encoder.encode(text, convert_to_tensor=True).to(device)

generated_text = vec_to_text(
    embedding, decoder, tokenizer, prenet, 30
)

print(generated_text)

y, what's your name ?                       


In [6]:
text = "Today is a very sunny day indeed."
embedding = encoder.encode(text, convert_to_tensor=True).to(device)

generated_text = vec_to_text(
    embedding, decoder, tokenizer, prenet, 30
)

print(generated_text)

 exact day of sunshine today .                        


In [32]:
from sklearn.metrics.pairwise import cosine_similarity

# Get sentence embeddings (assumes encoder is from sentence-transformers or similar)
vec1 = encoder.encode("Let's get a pizza tonight.", convert_to_numpy=True)
vec2 = encoder.encode("Let's have a pizza tonight", convert_to_numpy=True)

# Compute cosine similarity
similarity = cosine_similarity([vec1], [vec2])

print(similarity)

[[0.9564296]]


In [1]:
import datasets

data = datasets.load_dataset("francescoortame/bookcorpus-rand-1M")

/anaconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [4]:
a = data['train'].train_test_split(0.1, seed=42)